In [1]:
from add_label import buildBST, getGRCIndex

In [2]:
import gzip
import json
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix

In [3]:
with gzip.open('NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar.json.gz','rb') as f:
    design = json.loads(f.read().decode('utf-8'))

In [4]:
instances = pd.DataFrame(design['instances'])
nets = pd.DataFrame(design['nets'])

In [5]:
conn=np.load('NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar_connectivity.npz')
A = coo_matrix((conn['data'], (conn['row'], conn['col'])), shape=conn['shape'])
A = A.__mul__(A.T)

In [6]:
instances

,name,id,xloc,yloc,cell,orient
0,clk_gate_out_reg/latch,0,41984,44544,23,0
1,clk_gate_out_reg_0/latch,1,41984,47616,23,6
2,clk_gate_out_reg_1/latch,2,44160,44544,23,0
3,clk_gate_out_reg_2/latch,3,44160,47616,23,0
4,clk_gate_out_reg_3/latch,4,46336,47616,23,0
...,...,...,...,...,...,...
3947,U4123,3947,21888,53760,42,4
3948,U4125,3948,33664,66048,42,0
3949,U4128,3949,23296,66048,34,0
3950,ZCTSBUF_205_132,3950,40576,44544,11,0


In [7]:
def buildBST(array,start=0,finish=-1):
    if finish<0:
        finish = len(array)
    mid = (start + finish) // 2
    if mid-start==1:
        ltl=start
    else:
        ltl=buildBST(array,start,mid)
    
    if finish-mid==1:
        gtl=mid
    else:
        gtl=buildBST(array,mid,finish)
        
    return((array[mid],ltl,gtl))

In [8]:
congestion_data = np.load('NCSU-DigIC-GraphData-2023-07-25/xbar/1/xbar_congestion.npz')
xbst = buildBST(congestion_data['xBoundaryList'])
ybst = buildBST(congestion_data['yBoundaryList'])
demand = np.zeros(shape = [instances.shape[0],])

In [9]:
def getGRCIndex(x,y,xbst,ybst):
    while (type(xbst)==tuple):
        if x < xbst[0]:
            xbst=xbst[1]
        else:
            xbst=xbst[2]
            
    while (type(ybst)==tuple):
        if y < ybst[0]:
            ybst=ybst[1]
        else:
            ybst=ybst[2]
            
    return ybst, xbst

In [10]:
for k in range(instances.shape[0]):
    # print(k)
    xloc = instances.iloc[k]['xloc']; yloc = instances.iloc[k]['yloc']
    i,j=getGRCIndex(xloc,yloc,xbst,ybst)
    d = 0 
    for l in list(congestion_data['layerList']): 
        lyr=list(congestion_data['layerList']).index(l)
        d += congestion_data['demand'][lyr][i][j]
    demand[k] = d

In [11]:
instances['routing_demand'] = demand

## Question 1a

In [12]:
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data

In [13]:
# Use randomly selected 70% instances as training and 30% instances as test
np.random.seed(42)
train, test= train_test_split(instances, test_size=0.3, random_state=42)
test = test.reset_index()

In [14]:
node_features = torch.tensor(train[['xloc', 'yloc', 'cell', 'orient']].values, dtype=torch.float)
num_nodes = len(node_features)
edge_index = torch.tensor([[i for i in range(num_nodes)], [i for i in range(num_nodes)]], dtype=torch.long)
data = Data(x=node_features, edge_index=edge_index)

In [15]:
model = GCNConv(in_channels=4, hidden_channels=64, out_channels=1)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [16]:
num_epochs = 5000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Forward pass
    output = model(data.x, data.edge_index)
    
    # Assuming you have labels (target values) for a supervised task
    # Replace 'labels_column' with the actual column containing your labels
    labels = torch.tensor(train['routing_demand'].values, dtype=torch.float)
    
    # Compute the loss
    loss = criterion(output, labels)
    
    # Backward pass
    loss.backward()
    
    # Update weights
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

/Users/lisahwang/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([2766])) that is different to the input size (torch.Size([2766, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 1/5000, Loss: 1319650944.0
Epoch 1001/5000, Loss: 866.3240966796875
Epoch 2001/5000, Loss: 713.2647705078125
Epoch 3001/5000, Loss: 508.3110046386719
Epoch 4001/5000, Loss: 298.0909729003906


In [17]:
def testmodel():
    model.eval()
    output = model(node_features, edge_index)
    loss_test = criterion(output[test.index], labels[test.index])
    print("Test set results:",
          "MSE loss= {:.4f}".format(loss_test.item()))

In [18]:
testmodel()

Test set results: MSE loss= 145.4440


/Users/lisahwang/opt/anaconda3/envs/dsc80/lib/python3.8/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1186])) that is different to the input size (torch.Size([1186, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


In [19]:
num_hidden_layers = 2
hidden_dimensions = [64, 32]

class GCNModel(nn.Module):
    def __init__(self, in_channels, hidden_dimensions, out_channels):
        super(GCNModel, self).__init__()
        self.layers = nn.ModuleList()
        
        # Add input layer
        self.layers.append(GCNConv(in_channels, hidden_dimensions[0]))
        
        # Add hidden layers
        for i in range(1, num_hidden_layers):
            self.layers.append(GCNConv(hidden_dimensions[i-1], hidden_dimensions[i]))
        
        # Add output layer
        self.layers.append(GCNConv(hidden_dimensions[-1], out_channels))

    def forward(self, x, edge_index):
        for layer in self.layers:
            x = layer(x, edge_index)
        return x

# Instantiate the model
model = GCNModel(in_channels=4, hidden_dimensions=hidden_dimensions, out_channels=1)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 5000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Forward pass
    output = model(data.x, data.edge_index)
    
    # Assuming you have labels (target values) for a supervised task
    # Replace 'labels_column' with the actual column containing your labels
    labels = torch.tensor(train['routing_demand'].values, dtype=torch.float)
    
    # Compute the loss
    loss = criterion(output, labels)
    
    # Backward pass
    loss.backward()
    
    # Update weights
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

def testmodel():
    model.eval()
    output = model(node_features, edge_index)
    loss_test = criterion(output[test.index], labels[test.index])
    print("Test set results:",
          "MSE loss= {:.4f}".format(loss_test.item()))

testmodel()

Epoch 1/5000, Loss: 242283440.0
Epoch 1001/5000, Loss: 72.3841552734375
Epoch 2001/5000, Loss: 70.42475891113281
Epoch 3001/5000, Loss: 68.26179504394531
Epoch 4001/5000, Loss: 2407.57763671875
Test set results: MSE loss= 67.5126


In [22]:
num_hidden_layers = 3  # Change this to the desired number of hidden layers
hidden_dimensions = [64, 32, 16]  # Example: Three hidden layers with 64, 32, and 16 hidden units

# Build the GCN model with multiple layers
class GCNModel(nn.Module):
    def __init__(self, in_channels, hidden_dimensions, out_channels):
        super(GCNModel, self).__init__()
        self.layers = nn.ModuleList()
        
        # Add input layer
        self.layers.append(GCNConv(in_channels, hidden_dimensions[0]))
        
        # Add hidden layers
        for i in range(1, num_hidden_layers):
            self.layers.append(GCNConv(hidden_dimensions[i-1], hidden_dimensions[i]))
        
        # Add output layer
        self.layers.append(GCNConv(hidden_dimensions[-1], out_channels))

    def forward(self, x, edge_index):
        for layer in self.layers:
            x = layer(x, edge_index)
        return x

# Instantiate the model
model = GCNModel(in_channels=4, hidden_dimensions=hidden_dimensions, out_channels=1)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 5000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Forward pass
    output = model(data.x, data.edge_index)
    
    # Assuming you have labels (target values) for a supervised task
    # Replace 'labels_column' with the actual column containing your labels
    labels = torch.tensor(train['routing_demand'].values, dtype=torch.float)
    
    # Compute the loss
    loss = criterion(output, labels)
    
    # Backward pass
    loss.backward()
    
    # Update weights
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

def testmodel():
    model.eval()
    output = model(node_features, edge_index)
    loss_test = criterion(output[test.index], labels[test.index])
    print("Test set results:",
          "MSE loss= {:.4f}".format(loss_test.item()))

testmodel()

Epoch 1/5000, Loss: 93479648.0
Epoch 1001/5000, Loss: 75.83006286621094
Epoch 2001/5000, Loss: 74.41992950439453
Epoch 3001/5000, Loss: 72.35708618164062
Epoch 4001/5000, Loss: 69.81207275390625
Test set results: MSE loss= 69.0787


In [23]:
num_hidden_layers = 4  # Change this to the desired number of hidden layers
hidden_dimensions = [64, 32, 16, 8]  # Example: Three hidden layers with 64, 32, and 16 hidden units

# Build the GCN model with multiple layers
class GCNModel(nn.Module):
    def __init__(self, in_channels, hidden_dimensions, out_channels):
        super(GCNModel, self).__init__()
        self.layers = nn.ModuleList()
        
        # Add input layer
        self.layers.append(GCNConv(in_channels, hidden_dimensions[0]))
        
        # Add hidden layers
        for i in range(1, num_hidden_layers):
            self.layers.append(GCNConv(hidden_dimensions[i-1], hidden_dimensions[i]))
        
        # Add output layer
        self.layers.append(GCNConv(hidden_dimensions[-1], out_channels))

    def forward(self, x, edge_index):
        for layer in self.layers:
            x = layer(x, edge_index)
        return x

# Instantiate the model
model = GCNModel(in_channels=4, hidden_dimensions=hidden_dimensions, out_channels=1)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 5000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Forward pass
    output = model(data.x, data.edge_index)
    
    # Assuming you have labels (target values) for a supervised task
    # Replace 'labels_column' with the actual column containing your labels
    labels = torch.tensor(train['routing_demand'].values, dtype=torch.float)
    
    # Compute the loss
    loss = criterion(output, labels)
    
    # Backward pass
    loss.backward()
    
    # Update weights
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

def testmodel():
    model.eval()
    output = model(node_features, edge_index)
    loss_test = criterion(output[test.index], labels[test.index])
    print("Test set results:",
          "MSE loss= {:.4f}".format(loss_test.item()))

testmodel()

Epoch 1/5000, Loss: 668022784.0
Epoch 1001/5000, Loss: 73.16777038574219
Epoch 2001/5000, Loss: 72.20680236816406
Epoch 3001/5000, Loss: 70.7574462890625
Epoch 4001/5000, Loss: 68.8647232055664
Test set results: MSE loss= 68.5035


In [24]:
num_hidden_layers = 5  # Change this to the desired number of hidden layers
hidden_dimensions = [64, 32, 16, 8, 4]  # Example: Three hidden layers with 64, 32, and 16 hidden units

# Build the GCN model with multiple layers
class GCNModel(nn.Module):
    def __init__(self, in_channels, hidden_dimensions, out_channels):
        super(GCNModel, self).__init__()
        self.layers = nn.ModuleList()
        
        # Add input layer
        self.layers.append(GCNConv(in_channels, hidden_dimensions[0]))
        
        # Add hidden layers
        for i in range(1, num_hidden_layers):
            self.layers.append(GCNConv(hidden_dimensions[i-1], hidden_dimensions[i]))
        
        # Add output layer
        self.layers.append(GCNConv(hidden_dimensions[-1], out_channels))

    def forward(self, x, edge_index):
        for layer in self.layers:
            x = layer(x, edge_index)
        return x

# Instantiate the model
model = GCNModel(in_channels=4, hidden_dimensions=hidden_dimensions, out_channels=1)

criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 5000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Forward pass
    output = model(data.x, data.edge_index)
    
    # Assuming you have labels (target values) for a supervised task
    # Replace 'labels_column' with the actual column containing your labels
    labels = torch.tensor(train['routing_demand'].values, dtype=torch.float)
    
    # Compute the loss
    loss = criterion(output, labels)
    
    # Backward pass
    loss.backward()
    
    # Update weights
    optimizer.step()

    if epoch % 1000 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

def testmodel():
    model.eval()
    output = model(node_features, edge_index)
    loss_test = criterion(output[test.index], labels[test.index])
    print("Test set results:",
          "MSE loss= {:.4f}".format(loss_test.item()))

testmodel()

Epoch 1/5000, Loss: 226736464.0
Epoch 1001/5000, Loss: 93.14885711669922
Epoch 2001/5000, Loss: 88.8634033203125
Epoch 3001/5000, Loss: 82.4587173461914
Epoch 4001/5000, Loss: 74.42916870117188
Test set results: MSE loss= 68.4683


### My implementation of this model shows that 2 layers has the lowest MSE.

## Question 1b

In [36]:
class GCNWithAttention(nn.Module):
    def __init__(self, in_channels, out_channels, num_heads=2):
        super(GCNWithAttention, self).__init__()
        self.conv = GCNConv(in_channels, out_channels)
        self.attention = nn.MultiheadAttention(out_channels, num_heads=num_heads)

    def forward(self, x, edge_index):
        # Perform GCN message passing
        x = self.conv(x, edge_index)
        
        # Transpose x to match the expected shape for attention
        x = x.unsqueeze(0)  # Add a batch dimension
        x = x.transpose(0, 1)  # Swap batch and sequence dimensions
        
        # Compute attention scores and apply attention
        x, _ = self.attention(x, x, x)
        
        # Transpose back to the original shape
        x = x.transpose(0, 1).squeeze(0)
        
        x = F.relu(x)  # Apply non-linearity
        
        return x

# Instantiate the model
in_channels = 4  # Change this based on the number of input features
out_channels = 2766  # Change this based on the desired output dimension
num_heads = 2
model = GCNWithAttention(in_channels, out_channels, num_heads)

# Assuming you have a data instance named data
output = model(data.x, data.edge_index)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Assuming 'train_data' contains the node features and edge indices
data = Data(x=torch.tensor(train[['xloc', 'yloc', 'cell', 'orient']].values, dtype=torch.float),
            edge_index=torch.tensor([[i for i in range(num_nodes)], [i for i in range(num_nodes)]], dtype=torch.long))

# Convert labels to tensor
labels = torch.tensor(train['routing_demand'].values, dtype=torch.float)

# Training loop
num_epochs = 5000
for epoch in range(num_epochs):
    optimizer.zero_grad()
    
    # Forward pass
    output = model(data.x, data.edge_index)
    
    # Compute the loss
    loss = criterion(output, labels)
    
    # Backward pass
    loss.backward()
    
    # Update weights
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

Epoch 1/5000, Loss: 418328.0


KeyboardInterrupt: 